In [2]:
import rasterio
import rasterstats
from stac import STAC

In [3]:
from rasterstats import zonal_stats, point_query

In [4]:
stac_client = STAC('http://brazildatacube.dpi.inpe.br/bdc-stac/0.8.0/')

In [5]:
collection = stac_client.collection('C4_64_16D_MED')

In [6]:
items = collection.get_items()

In [7]:
red = items.features[0].assets['red'].download()

In [8]:
rasterio.open(red).read(1)

array([[413, 418, 420, ..., 817, 729, 706],
       [420, 423, 422, ..., 831, 761, 702],
       [423, 414, 406, ..., 853, 786, 698],
       ...,
       [774, 808, 703, ..., 717, 717, 629],
       [768, 775, 672, ..., 755, 721, 661],
       [685, 627, 586, ..., 745, 714, 669]], dtype=int16)

In [9]:
stats = zonal_stats('data/br-uf/BRUFE250GC_SIR.shp', red)

In [10]:
from pprint import pprint
pprint(stats)

[{'count': 0, 'max': None, 'mean': None, 'min': None},
 {'count': 0, 'max': None, 'mean': None, 'min': None},
 {'count': 0, 'max': None, 'mean': None, 'min': None},
 {'count': 0, 'max': None, 'mean': None, 'min': None},
 {'count': 0, 'max': None, 'mean': None, 'min': None},
 {'count': 0, 'max': None, 'mean': None, 'min': None},
 {'count': 0, 'max': None, 'mean': None, 'min': None},
 {'count': 0, 'max': None, 'mean': None, 'min': None},
 {'count': 0, 'max': None, 'mean': None, 'min': None},
 {'count': 0, 'max': None, 'mean': None, 'min': None},
 {'count': 0, 'max': None, 'mean': None, 'min': None},
 {'count': 0, 'max': None, 'mean': None, 'min': None},
 {'count': 0, 'max': None, 'mean': None, 'min': None},
 {'count': 0, 'max': None, 'mean': None, 'min': None},
 {'count': 0, 'max': None, 'mean': None, 'min': None},
 {'count': 0, 'max': None, 'mean': None, 'min': None},
 {'count': 0, 'max': None, 'mean': None, 'min': None},
 {'count': 0, 'max': None, 'mean': None, 'min': None},
 {'count':